<a href="https://colab.research.google.com/github/maheshkulkarni123OS/OperatingSystemByMaheshKulkarni/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sn6qxew3
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sn6qxew3
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=35ff992d00c8a60ed3da743484f3e7827b88bee9ce6446b735c13f7cfd931875
  Stored in directory: /tmp/pip-ephem-wheel-cache-r2gjjln8/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [0]:
%%cu
#ifdef __INTELLISENSE__
void __syncthreads();
#endif

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdlib.h>
#include <stdio.h>
#include <iostream>
#include <chrono>

using namespace std;
using namespace std::chrono;

#define WSIZE 32
#define LOOPS 100
#define UPPER_BIT 10
#define LOWER_BIT 0

__device__ unsigned int ddata[WSIZE];
__device__ int ddata_s[WSIZE];

template <typename T, unsigned S>
inline unsigned arraysize(const T(&v)[S])
{
	return S;
}

template<typename T>
void printArray(T &arr)
{
	for (int i = 0; i < arraysize(arr); ++i)
	{
		cout << "Array[" << i << "]: " << *(arr + i) << endl;
	}
}

template<typename T>
void fillArray(T &arr)
{
	srand(time(NULL));
	for (int i = 0; i < arraysize(arr); ++i)
	{
		arr[i] = rand() % 1024;
	}
}

void print(int arr[], int n)
{
	for (int i = 0; i < n; i++)
	{
		cout << arr[i] << " ";
	}
	cout << endl;
}

template<typename T>
T findMax(T &arr)
{
	T max = 0;
	for (int i = 0; i < arraysize(arr); ++i)
	{
		if (arr[i] > max)
		{
			max = arr[i];
		}
	}
	return max;
}

__device__ int getMax(int arr[], int n)
{
	int mx = arr[0];
	for (int i = 1; i < n; i++)
		if (arr[i] > mx)
			mx = arr[i];
	return mx;
}

__device__ void countSort(int arr[], int n, int exp)
{
	int output[1024]; // Output array
	int i, count[10] = { 0 };

	// Store count of occurrences in count[]
	for (i = 0; i < n; i++)
		count[(arr[i] / exp) % 10]++;

	// Change count[i] so that count[i] now contains actual
	// position of this digit in output[]
	for (i = 1; i < 10; i++)
	{
		count[i] += count[i - 1];
	}

	// Build the output array
	for (i = n - 1; i >= 0; i--)
	{
		output[count[(arr[i] / exp) % 10] - 1] = arr[i];
		count[(arr[i] / exp) % 10]--;
	}

	// Copy the output array to arr[], so that arr[] now
	// contains sorted numbers according to current digit
	for (i = 0; i < n; i++)
		arr[i] = output[i];
}

__device__ void radixsort(int arr[], int n)
{
	// Find the maximum number to know number of digits
	int m = getMax(arr, n);

	// Do counting sort for every digit. Note that instead
	// of passing digit number, exp is passed. exp is 10^i
	// where i is current digit number
	for (int exp = 1; m / exp > 0; exp *= 10)
		countSort(arr, n, exp);
}

__global__ void serialRadix()
{
	radixsort(ddata_s, WSIZE);
	__syncthreads();
}

__global__ void parallelRadix()
{
	// This data in shared memory
	__shared__ volatile unsigned int sdata[WSIZE * 2];

	// Load from global into shared variable
	sdata[threadIdx.x] = ddata[threadIdx.x];

	unsigned int bitmask = 1 << LOWER_BIT;
	unsigned int offset = 0;
	// -1, -2, -4, -8, -16, -32, -64, -128, -256,...
	unsigned int thrmask = 0xFFFFFFFFU << threadIdx.x;
	unsigned int mypos;

	// For each LSB to MSB
	for (int i = LOWER_BIT; i <= UPPER_BIT; i++)
	{
		unsigned int mydata = sdata[((WSIZE - 1) - threadIdx.x) + offset];
		unsigned int mybit = mydata&bitmask;
		// Get population of ones and zeroes
		unsigned int ones = __ballot(mybit);
		unsigned int zeroes = ~ones;
		// Switch ping-pong buffers
		offset ^= WSIZE;

		// Do zeroes, then ones
		if (!mybit)
		{
			mypos = __popc(zeroes&thrmask);
		}
		else  {      // Threads with a one bit
			// Get my position in ping-pong buffer
			mypos = __popc(zeroes) + __popc(ones&thrmask);
		}

		// Move to buffer  (or use shfl for cc 3.0)
		sdata[mypos - 1 + offset] = mydata;
		// Repeat for next bit
		bitmask <<= 1;
	}
	// Put results to global
	ddata[threadIdx.x] = sdata[threadIdx.x + offset];
}

int main() {

	/* Parallel Radix Sort */

	unsigned int hdata[WSIZE];
	float totalTime = 0;

	for (int lcount = 0; lcount < LOOPS; lcount++)
	{
		srand(time(NULL));
		
		unsigned int range = 1U << UPPER_BIT;

		// Fill array with random elements
		// Range = 1024
		for (int i = 0; i < WSIZE; i++)
		{
			hdata[i] = i;
		}

		
		cudaMemcpyToSymbol(ddata, hdata, WSIZE * sizeof(unsigned int));

		
		high_resolution_clock::time_point t1 = high_resolution_clock::now();
		parallelRadix <<< 1, WSIZE >>>();
		// Make kernel function synchronous
		cudaDeviceSynchronize();
		// Execution time measurement, that point stops the clock
		high_resolution_clock::time_point t2 = high_resolution_clock::now();

		// Execution time measurement, that is the result
		auto duration = duration_cast<milliseconds>(t2 - t1).count();

		// Summination of each loops' execution time
		totalTime += (float)duration / 1000.00;

		// Copy data from device to host
		cudaMemcpyFromSymbol(hdata, ddata, WSIZE * sizeof(unsigned int));
	}

	printf("Parallel Radix Sort:\n");
	printf("Array size = %d\n", WSIZE * LOOPS);
	printf("Time elapsed = %fseconds\n", totalTime);

	/* Serial Radix Sort */

	unsigned int hdata_s[WSIZE];
	totalTime = 0;

	for (int lcount = 0; lcount < LOOPS; lcount++)
	{
		srand(time(NULL));
		// Array elements have value in range of 1024
		unsigned int range = 1U << UPPER_BIT;

		// Fill array with random elements
		// Range = 1024
		for (int i = 0; i < WSIZE; i++)
		{
			hdata_s[i] = i;
		}

		// Copy data from host to device
		cudaMemcpyToSymbol(ddata_s, hdata_s, WSIZE * sizeof(unsigned int));

		// Execution time measurement, that point starts the clock
		high_resolution_clock::time_point t1 = high_resolution_clock::now();
		serialRadix <<< 1, 1 >>>();
		// Make kernel function synchronous
		cudaDeviceSynchronize();
		// Execution time measurement, that point stops the clock
		high_resolution_clock::time_point t2 = high_resolution_clock::now();

		// Execution time measurement, that is the result
		auto duration = duration_cast<milliseconds>(t2 - t1).count();

		// Summination of each loops' execution time
		totalTime += (float)duration / 1000.00;

		// Copy data from device to host
		cudaMemcpyFromSymbol(hdata_s, ddata_s, WSIZE * sizeof(unsigned int));
	}

	printf("\nSerial Radix Sort:\n");
	printf("Array size = %d\n", WSIZE * LOOPS);
	printf("Time elapsed = %fseconds\n\n", totalTime);

	return 0;
}

Parallel Radix Sort:
Array size = 3200
Time elapsed = 0.000000seconds

Serial Radix Sort:
Array size = 3200
Time elapsed = 0.006000seconds




In [0]:


%%cu
#include<stdio.h>
__global__ void kernel(void){
		printf("Hello World");

}
int main(void)
{
	kernel<<< 1,1 >>>();
	printf("Hello World!\n");
	return 0;
}

Hello World!



In [0]:
#Assignment 5
%%cu
#include <pthread.h>
#include <stdio.h>

const int N = 1 << 20;

__global__ void kernel(float *x, int n)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    printf("Tid = %d",tid);
   
}

void *smaa(void *dummy)
{
    float *data;
    static int cnt=0;
    cudaMalloc(&data, N * sizeof(float));
    printf("\nLaunch Kernel %d",cnt);
    cnt++;
    kernel<<<1, 32>>>(data, N);

    cudaStreamSynchronize(0);

    return NULL;
}

int main()
{
    const int num_threads = 8;

    pthread_t threads[num_threads];

    for (int i = 0; i < num_threads; i++) {
        if (pthread_create(&threads[i], NULL, smaa, 0)) {
            fprintf(stderr, "Error creating threadn");
            return 1;
        }
    }

    for (int i = 0; i < num_threads; i++) {
        if(pthread_join(threads[i], NULL)) {
            fprintf(stderr, "Error joining threadn");
            return 2;
        }
    }

    cudaDeviceReset();

    return 0;
}



Launch Kernel 0
Launch Kernel 1Tid = 0Tid = 1Tid = 2Tid = 3Tid = 4Tid = 5Tid = 6Tid = 7Tid = 8Tid = 9Tid = 10Tid = 11Tid = 12Tid = 13Tid = 14Tid = 15Tid = 16Tid = 17Tid = 18Tid = 19Tid = 20Tid = 21Tid = 22Tid = 23Tid = 24Tid = 25Tid = 26Tid = 27Tid = 28Tid = 29Tid = 30Tid = 31
Launch Kernel 2Tid = 0Tid = 1Tid = 2Tid = 3Tid = 4Tid = 5Tid = 6Tid = 7Tid = 8Tid = 9Tid = 10Tid = 11Tid = 12Tid = 13Tid = 14Tid = 15Tid = 16Tid = 17Tid = 18Tid = 19Tid = 20Tid = 21Tid = 22Tid = 23Tid = 24Tid = 25Tid = 26Tid = 27Tid = 28Tid = 29Tid = 30Tid = 31
Launch Kernel 3Tid = 0Tid = 1Tid = 2Tid = 3Tid = 4Tid = 5Tid = 6Tid = 7Tid = 8Tid = 9Tid = 10Tid = 11Tid = 12Tid = 13Tid = 14Tid = 15Tid = 16Tid = 17Tid = 18Tid = 19Tid = 20Tid = 21Tid = 22Tid = 23Tid = 24Tid = 25Tid = 26Tid = 27Tid = 28Tid = 29Tid = 30Tid = 31
Launch Kernel 3Tid = 0Tid = 1Tid = 2Tid = 3Tid = 4Tid = 5Tid = 6Tid = 7Tid = 8Tid = 9Tid = 10Tid = 11Tid = 12Tid = 13Tid = 14Tid = 15Tid = 16Tid = 17Tid = 18Tid = 19Tid = 20Tid = 21Tid = 22Tid = 23

In [0]:
#Assignment Number 2
#1]Vector addition using multiple block single thread
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
__global__ void add(int *a, int *b, int *c)
{
  c[blockIdx.x]=a[blockIdx.x]+b[blockIdx.x];
}
#define N 512
int main(void)
{
  int *a, *b,*c;
  int i;
  int *dev_a, *dev_b, *dev_c;
  int size=N*sizeof(int);
  cudaMalloc((void**)&dev_a,size);
  cudaMalloc((void**)&dev_b,size);
  cudaMalloc((void**)&dev_c,size);
  a=(int*)malloc(size);
  b=(int*)malloc(size);
  c=(int*)malloc(size);
  

  for(i=0;i<N;i++)
  {
    a[i]=rand()%10;
    b[i]=rand()%10;
  }

  cudaMemcpy(dev_a,a,size,cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b,b,size,cudaMemcpyHostToDevice);
  add<<<N,1>>>(dev_a,dev_b,dev_c);
  cudaMemcpy(c,dev_c,size,cudaMemcpyDeviceToHost);
  for(i=0;i<N;i++)
    printf("%d ",c[i]);
  free(a);
  free(b);
  free(c);
  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);
return 0;
}


9 12 8 8 10 9 9 9 6 8 9 16 2 5 12 11 10 16 9 3 12 10 11 12 5 9 1 9 2 7 10 11 11 15 10 9 7 11 8 3 15 14 12 4 13 2 14 11 12 13 5 12 8 9 9 8 1 7 14 13 9 8 8 6 16 13 10 11 9 13 9 11 12 8 13 9 9 13 7 6 17 12 1 3 8 8 13 4 12 15 3 15 12 5 13 16 8 16 4 17 10 6 11 6 12 16 16 13 7 3 9 7 4 14 10 11 6 14 9 9 10 12 14 2 1 6 1 13 6 10 7 14 10 11 6 6 11 9 9 4 1 7 3 10 15 12 11 5 12 4 4 8 13 11 17 9 10 18 6 9 7 5 6 18 5 10 7 13 7 12 4 11 7 4 4 2 10 11 2 11 8 4 10 15 7 8 18 9 5 2 13 8 8 7 5 15 6 8 5 10 17 10 16 14 11 6 11 4 13 17 6 15 9 6 10 5 11 8 13 16 6 5 4 15 8 8 11 12 11 11 1 15 10 9 11 12 13 13 14 8 7 9 18 15 7 7 9 2 10 8 3 7 4 10 5 13 11 6 11 12 13 8 6 10 8 17 6 7 1 4 14 8 9 2 11 9 13 6 3 15 14 4 13 6 16 14 15 8 11 10 12 15 11 10 9 16 13 7 8 11 9 16 12 6 10 3 2 3 12 8 5 8 2 7 4 13 7 8 10 10 8 9 2 11 12 10 9 16 11 10 7 6 13 12 5 9 6 9 17 11 7 4 7 7 9 9 7 7 16 7 17 2 11 11 8 6 11 6 8 13 8 8 4 2 8 12 5 5 11 11 14 9 13 10 7 5 11 8 3 8 10 4 9 4 5 8 10 0 7 11 11 7 6 7 14 8 11 5 9 7 4 9 8 12 7 17 4 6 1

In [0]:
#2]Vector Addition using single thread single block 

%%cu
#include<stdio.h>
__global__ void add(int *a, int *b, int *c)
{
  *c= *a + *b;
}

int main(void)
{
int a,b,c;
int *dev_a,*dev_b,*dev_c;
int size=sizeof(int);
  cudaMalloc((void **)&dev_a,size);
  cudaMalloc((void **)&dev_b,size);
  cudaMalloc((void **)&dev_c,size);
  a=2;
  b=7;
  cudaMemcpy(dev_a,&a,size,cudaMemcpyHostToDevice);
cudaMemcpy(dev_b,&b,size,cudaMemcpyHostToDevice);
  add<<< 1,1 >>>(dev_a,dev_b,dev_c);
  cudaMemcpy(&c,dev_c,size,cudaMemcpyDeviceToHost);
  printf("addition = %d",c);
  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);

return 0;

}


addition = 9


In [0]:
#]Vector Addition using multiple thread single block
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
#define N 512

__global__ void add(int *a, int *b, int *c ) {
c[threadIdx.x] = a[threadIdx.x] + b[ threadIdx.x];
}


int main( void ) {

int *a, *b, *c;
int *dev_a, *dev_b, *dev_c;
int size = N * sizeof( int ); 
cudaMalloc( (void**)&dev_a, size );
cudaMalloc( (void**)&dev_b, size );
cudaMalloc( (void**)&dev_c, size );
a = (int*)malloc( size );
b = (int*)malloc( size );
c = (int*)malloc( size );

for(int i=0;i<N;i++)
{
  a[i]=rand()%10;
  b[i]=rand()%10;
}
cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice );
cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice );
add<<< 1,N >>>( dev_a, dev_b, dev_c );
cudaMemcpy( c, dev_c, size, cudaMemcpyDeviceToHost );
for(int i=0;i<N;i++)
printf("%d ",c[i]);
free( a ); 
free( b );
free( c );
cudaFree(dev_a );
cudaFree(dev_b );
cudaFree(dev_c );
return 0;
}


9 12 8 8 10 9 9 9 6 8 9 16 2 5 12 11 10 16 9 3 12 10 11 12 5 9 1 9 2 7 10 11 11 15 10 9 7 11 8 3 15 14 12 4 13 2 14 11 12 13 5 12 8 9 9 8 1 7 14 13 9 8 8 6 16 13 10 11 9 13 9 11 12 8 13 9 9 13 7 6 17 12 1 3 8 8 13 4 12 15 3 15 12 5 13 16 8 16 4 17 10 6 11 6 12 16 16 13 7 3 9 7 4 14 10 11 6 14 9 9 10 12 14 2 1 6 1 13 6 10 7 14 10 11 6 6 11 9 9 4 1 7 3 10 15 12 11 5 12 4 4 8 13 11 17 9 10 18 6 9 7 5 6 18 5 10 7 13 7 12 4 11 7 4 4 2 10 11 2 11 8 4 10 15 7 8 18 9 5 2 13 8 8 7 5 15 6 8 5 10 17 10 16 14 11 6 11 4 13 17 6 15 9 6 10 5 11 8 13 16 6 5 4 15 8 8 11 12 11 11 1 15 10 9 11 12 13 13 14 8 7 9 18 15 7 7 9 2 10 8 3 7 4 10 5 13 11 6 11 12 13 8 6 10 8 17 6 7 1 4 14 8 9 2 11 9 13 6 3 15 14 4 13 6 16 14 15 8 11 10 12 15 11 10 9 16 13 7 8 11 9 16 12 6 10 3 2 3 12 8 5 8 2 7 4 13 7 8 10 10 8 9 2 11 12 10 9 16 11 10 7 6 13 12 5 9 6 9 17 11 7 4 7 7 9 9 7 7 16 7 17 2 11 11 8 6 11 6 8 13 8 8 4 2 8 12 5 5 11 11 14 9 13 10 7 5 11 8 3 8 10 4 9 4 5 8 10 0 7 11 11 7 6 7 14 8 11 5 9 7 4 9 8 12 7 17 4 6 1

In [0]:
#4]Vector addition using multiple threads multiple blocks
%%cu
#include<stdio.h>
#include<time.h>
#define N (64*64)
#define THREADS_PER_BLOCK 512

__global__ void add(int *a, int *b, int *c ) {
int index = threadIdx.x + blockIdx.x * blockDim.x;
c[index] = a[index] + b[index];
}

int main( void ) {
int *a, *b, *c;
int *dev_a, *dev_b, *dev_c;
int size = N * sizeof( int ); 
cudaMalloc( (void**)&dev_a, size );
cudaMalloc( (void**)&dev_b, size );
cudaMalloc( (void**)&dev_c, size );
a = (int*)malloc( size );
b = (int*)malloc( size );
c = (int*)malloc( size );

for(int i=0;i<N;i++)
{
a[i]=rand()%10;
b[i]=rand()%10;
}
cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice );
cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice );
add<<< N/THREADS_PER_BLOCK, THREADS_PER_BLOCK >>>( dev_a, dev_b, dev_c );
cudaMemcpy( c, dev_c, size, cudaMemcpyDeviceToHost );

for(int i=0;i<N;i++)
{
  printf("%d ",c[i]);
}

free( a ); free( b ); free( c );
cudaFree(dev_a );
cudaFree(dev_b );
cudaFree(dev_c );
return 0;
}


9 12 8 8 10 9 9 9 6 8 9 16 2 5 12 11 10 16 9 3 12 10 11 12 5 9 1 9 2 7 10 11 11 15 10 9 7 11 8 3 15 14 12 4 13 2 14 11 12 13 5 12 8 9 9 8 1 7 14 13 9 8 8 6 16 13 10 11 9 13 9 11 12 8 13 9 9 13 7 6 17 12 1 3 8 8 13 4 12 15 3 15 12 5 13 16 8 16 4 17 10 6 11 6 12 16 16 13 7 3 9 7 4 14 10 11 6 14 9 9 10 12 14 2 1 6 1 13 6 10 7 14 10 11 6 6 11 9 9 4 1 7 3 10 15 12 11 5 12 4 4 8 13 11 17 9 10 18 6 9 7 5 6 18 5 10 7 13 7 12 4 11 7 4 4 2 10 11 2 11 8 4 10 15 7 8 18 9 5 2 13 8 8 7 5 15 6 8 5 10 17 10 16 14 11 6 11 4 13 17 6 15 9 6 10 5 11 8 13 16 6 5 4 15 8 8 11 12 11 11 1 15 10 9 11 12 13 13 14 8 7 9 18 15 7 7 9 2 10 8 3 7 4 10 5 13 11 6 11 12 13 8 6 10 8 17 6 7 1 4 14 8 9 2 11 9 13 6 3 15 14 4 13 6 16 14 15 8 11 10 12 15 11 10 9 16 13 7 8 11 9 16 12 6 10 3 2 3 12 8 5 8 2 7 4 13 7 8 10 10 8 9 2 11 12 10 9 16 11 10 7 6 13 12 5 9 6 9 17 11 7 4 7 7 9 9 7 7 16 7 17 2 11 11 8 6 11 6 8 13 8 8 4 2 8 12 5 5 11 11 14 9 13 10 7 5 11 8 3 8 10 4 9 4 5 8 10 0 7 11 11 7 6 7 14 8 11 5 9 7 4 9 8 12 7 17 4 6 1

In [0]:
#Assignment 3
#Matrix Multiplication-
%%cu
#define N 3
#include <stdio.h>

__global__ void matrixMult (int *a, int *b, int *c, int width)
{
int i, sum = 0;
int col = threadIdx.x + blockDim.x * blockIdx.x;
int row = threadIdx.y + blockDim.y * blockIdx.y;
if(col < width && row < width)
for (i = 0; i< width; i++)
{
sum += a[row * width + i] * b[i * width + col];
}
c[row * width + col] = sum;
}

int main()
{
int a[N][N], b[N][N], c[N][N];
int *dev_a, *dev_b, *dev_c;
 int i=1;
for (int y = 0; y < N; y++)
                          {
for (int x = 0; x < N; x++)
a[y][x]=i++;
                          }
i=9;
   for (int y = 0; y < N; y++)
        {
       for (int x = 0; x < N; x++)
                b[y][x]=i--;
         }

int size = N * N * sizeof(int);
cudaMalloc((void **) &dev_a, size);
cudaMalloc((void **) &dev_b, size);
cudaMalloc((void **) &dev_c, size);
cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);
dim3 dimGrid(1, 1);
dim3 dimBlock(N, N);
matrixMult<<<dimGrid, dimBlock>>>(dev_a, dev_b, dev_c, N);
cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);
cudaFree(dev_a);
cudaFree(dev_b);
cudaFree(dev_c);
for (int y = 0; y < N; y++) {
for (int x = 0; x < N; x++) {
printf("%d \t", c[y][x]);
}
printf("\n");
}
return 0;

}


30 	24 	18 	
84 	69 	54 	
138 	114 	90 	



In [0]:
#Assignment 3
#Dot product using shared memory-
%%cu
#include<stdio.h>
#define N (2048*2048)
#define THREADS_PER_BLOCK 512
__global__ void dot( int *a, int *b, int *c ) {
__shared__ int temp[THREADS_PER_BLOCK];
int index = threadIdx.x + blockIdx.x * blockDim.x;
temp[threadIdx.x] = a[index] * b[index];
__syncthreads();
if( 0 == threadIdx.x ) {
int sum = 0;
for( int i = 0; i < THREADS_PER_BLOCK; i++ )
sum += temp[i];
atomicAdd( c , sum );
}
}

int main( void ) {
int *a, *b, *c,i; 
int *dev_a, *dev_b, *dev_c;
int size = N * sizeof( int ); 
cudaMalloc( (void**)&dev_a, size );
cudaMalloc( (void**)&dev_b, size );
cudaMalloc( (void**)&dev_c, sizeof( int ) );
a = (int *)malloc( size );
b = (int *)malloc( size );
c = (int *)malloc( sizeof( int ) );
int sumtest= 0;
for(i=0;i<N;i++)
   {
       a[i] = rand() % 10;
       b[i] = rand() % 10;
       sumtest += a[i]*b[i];
   }
cudaMemcpy( dev_a, a, size, cudaMemcpyHostToDevice );
cudaMemcpy( dev_b, b, size, cudaMemcpyHostToDevice );
dot<<< N/THREADS_PER_BLOCK, THREADS_PER_BLOCK >>>( dev_a, dev_b, dev_c );
cudaMemcpy( c, dev_c, sizeof( int ) , cudaMemcpyDeviceToHost );
printf(" Using CPU:- %d\n ",sumtest);
printf("Using GPU:- %d",*c);
free( a );
free( b );
free( c );
cudaFree( dev_a );
cudaFree( dev_b );
cudaFree( dev_c );
return 0;
}


 Using CPU:- 85019351
 Using GPU:- 85019351


In [0]:
#Assignment 4
#Cuda Reduction-
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <iostream>
#include <numeric>
using namespace std;

__global__ void sum(int* input)
{
  const int tid = threadIdx.x;

  auto step_size = 1;
  int number_of_threads = blockDim.x;

  while (number_of_threads > 0)
  {
    if (tid < number_of_threads)
    {
      const auto fst = tid * step_size * 2;
      const auto snd = fst + step_size;
      input[fst] += input[snd];
    }

    step_size <<= 1; 
    number_of_threads >>= 1;
  }
}

int main()
{
  const auto count = 8;
  const int size = count * sizeof(int);
  int h[] = {13, 27, 15, 14, 33, 2, 24, 6};

  int* d;
  
  cudaMalloc(&d, size);
  cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

  sum <<<1, count / 2 >>>(d);

  int result;
  cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

  cout << "Sum is " << result << endl;

  getchar();

  cudaFree(d);
  delete[] h;

  return 0;
}


Sum is 134
src/tcmalloc.cc:283] Attempt to free invalid pointer 0x7ffce3aac940 

